In [4]:
!pip install langchain langchain-community sentence-transformers chromadb pdfminer.six faiss-cpu transformers torch

ERROR: Could not find a version that satisfies the requirement langchain (from versions: none)
ERROR: No matching distribution found for langchain


In [5]:
import os
import sys
import logging
from pathlib import Path

# LangChain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

# Transformers for running local models
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    pipeline
)

# Set up logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

print(f"Python version: {sys.version}")
print("All libraries imported successfully!")

ModuleNotFoundError: No module named 'langchain'